In [ ]:
!pip3 install geopy
!pip3 install numpy
!pip3 install datetime
!pip3 install h3

In [65]:
# @file createEpisode.ipynb
# @author Team GIS Gang
# @brief This notebook aims to generate Trip Episodes based on the given datasets of points

import pandas as pd
import os
import subprocess
import geopandas as gpd
import time
import datetime 
from os.path import join
import numpy as np
import h3
import geopy.distance
import geopy as gp

os.chdir("../src/")

#INPUT TRACE.CSV
#OUTPUT POINTS.CSV

# @param csv_path:
# @param title:
# @return an .csv file that consists of points
def createSegments(csv_path, title):

    #Reads csv and store in data frame
    df = pd.read_csv(csv_path)
    df = df[["lat","long","time"]]
    df['time']= pd.to_datetime(df['time'])
    df['index_column'] = df.index


    #Calculate stepsize using start and end time of dataset
    starttime = pd.to_datetime(df['time'].iloc[0])
    endtime = pd.to_datetime(df['time'].iloc[-1])
    totaltime = pd.Timedelta(endtime - starttime).seconds 
    stepsize = (round(len(df)/totaltime)) 

    #Trim points based on step size
    if(stepsize != 0):
        df = df.drop(df[df.index_column%stepsize !=0].index)
        df = df.reset_index()
    df = df[["lat","long","time"]]


    # Create Segment Directory
    try: 
        os.mkdir("Segment")
        print("Directory Segment Created ") 
    except FileExistsError:
        print("Directory Segment already exists")

    #Create trace directory in segment directory
    try: 
        path = "./Segment/"+title
        os.mkdir(path)
        print("Directory " , path ,  " Created ") 
    except FileExistsError:
        print("Directory " , path ,  " already exists")


    #Saves trip points to csv file 
    df.to_csv(path+"/points.csv", index=False)


#INPUT POINTS.CSV
#OUTPUT VELOCITIES.CSV
def createVelocities(csv_path):
    
    #Join the points cvs file by index n and n+1
    df = pd.read_csv(csv_path+"/points.csv")
    df2= df.iloc[:-1 , :]
    df1 = df.tail(-1)
    df1 = df1.reset_index()


    velocities = pd.DataFrame(columns= ["start_lat","start_long","end_lat","end_long","start_time","end_time","total_time","total_distance","velocity"])
    velocities["start_lat"] = df1["lat"]
    velocities["start_long"] = df1["long"]
    velocities["end_lat"] = df2["lat"]
    velocities["end_long"] = df2["long"]
    velocities["start_time"] = pd.to_datetime(df1['time'])
    velocities["end_time"] =  pd.to_datetime(df2['time'])
    velocities["total_time"] = (velocities["end_time"] - velocities["start_time"]).dt.total_seconds().abs()
    velocities['total_distance'] = velocities.apply(lambda x: gp.distance.distance((x[0], x[1]), (x[2], x[3])).m, axis=1)
    velocities['velocity'] = velocities['total_distance'] / (velocities['total_time'])

    #Saves trip segement to csv file
    velocities.to_csv(csv_path+"/velocities.csv", index=False)



from enum import Enum
class mode(Enum):
    STOP = 0
    WALK = 1
    DRIVE = 10


#INPUT VELOCITY.CSV
#OUTPUT EPISODE.CSV
def generateEpisodes(csv_path):

    df = pd.read_csv(csv_path+"/velocities.csv") 
    episode= pd.DataFrame(columns= ["start_lat","start_long","end_lat","end_long","start_time","end_time","mode"])

    startVel = df['velocity'].iloc[0] 
    startIndex = 0
    currMode = mode.STOP
    endIndex = 0

    if( startVel >= mode.WALK.value and startVel < mode.DRIVE.value):
        currMode = mode.WALK
    
    elif(startVel >= mode.DRIVE.value):
        currMode = mode.DRIVE

    for index in range(1,len(df)):

        prevIndex = index - 1
        endVel =  df['velocity'].iloc[index]
        endMode = currMode
        noChange = True
        
        starttime = pd.to_datetime(df['start_time'].iloc[startIndex])
        endtime = pd.to_datetime(df['end_time'].iloc[index])
        totaltime = pd.Timedelta(endtime - starttime).seconds 

        if( endVel >= mode.WALK.value and endVel < mode.DRIVE.value):
            endMode = mode.WALK
        elif(endVel >= mode.DRIVE.value):
            endMode = mode.DRIVE
        else: 
            endMode = mode.STOP

        if(currMode != endMode):

            new_row = [df['start_lat'].iloc[startIndex] ,
                df['start_long'].iloc[startIndex] ,
                df['end_lat'].iloc[index] ,
                df['end_long'].iloc[index] ,
                df['start_time'].iloc[startIndex],
                df['end_time'].iloc[index],
                currMode]    

            episode.loc[len(episode)] = new_row
            currMode = endMode
            startIndex = prevIndex


        if( index == len(df)-1 and noChange):
            new_row = [df['start_lat'].iloc[startIndex] ,
                df['start_long'].iloc[startIndex] ,
                df['end_lat'].iloc[index] ,
                df['end_long'].iloc[index] ,
                df['start_time'].iloc[startIndex],
                df['end_time'].iloc[index],
                currMode]    

            episode.loc[len(episode)] = new_row
            currMode = endMode
            startIndex = prevIndex


    episode.to_csv(csv_path+"/episode.csv", index=False)


def cleanEpisode(csv_path):
    episode = pd.read_csv(csv_path+"/episode.csv") 

    for index in range(len(episode)):
        starttime = pd.to_datetime(episode['start_time'].iloc[index]) 
        endtime = pd.to_datetime(episode['end_time'].iloc[index])
        timePassed = pd.Timedelta(endtime - starttime).seconds 
        if(episode['mode'].iloc[index] == mode.STOP and timePassed <60 and index !=len(episode)-1):
            print("hit")
            episode['start_time'].iloc[index+1]
            episode.drop(episode.index[index])

    episode.to_csv(csv_path+"/episode.csv", index=False)


# createSegments("../src/exampleDataset/trace_1.csv","trace1")
# createVelocities("./Segment/trace1")
# generateEpisodes("./Segment/trace1")
# cleanEpisode("./Segment/trace1")

# createSegments("../src/exampleDataset/trace_2.csv","trace2")
# createVelocities("./Segment/trace2")
# generateEpisodes("./Segment/trace2")
# cleanEpisode("./Segment/trace2")

# createSegments("../src/exampleDataset/trace_3.csv","trace3")
# createVelocities("./Segment/trace3")
# generateEpisodes("./Segment/trace3")
cleanEpisode("./Segment/trace3")




In [66]:

# createSegments("../src/exampleDataset/trace_1.csv","trace1")
# createVelocities("./Segment/trace1")
# generateEpisodes("./Segment/trace1")

# createSegments("../src/exampleDataset/trace_2.csv","trace2")
# createVelocities("./Segment/trace2")
# generateEpisodes("./Segment/trace2")

# createSegments("../src/exampleDataset/trace_3.csv","trace3")
# createVelocities("./Segment/trace3")


# targetPath = os.path.join("../", "SHPfiles/")
# os.mkdir(targetPath)
# os.chdir(targetPath)
# #Loops through each csvfile, creates point SHP file and writes to the new directory
# csvfile = "/Users/Nicholas/Desktop/Capstone-yoGERT/src/Segment/trace3/points.csv"
# print(csvfile)
# data = pd.read_csv(csvfile)
# dataGDF = gpd.GeoDataFrame(data, geometry = gpd.points_from_xy(data['long'], data['lat']))
# #Standardized ESRI wkt projection
# ESRI_WKT = 'GEOGCS["GCS_WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1984",6378137,298.257223563]],PRIMEM["Greenwich",0],UNIT["Degree",0.017453292519943295]]'
# shpfilename = "travelepisodew.shp"
# dataGDF.to_file(filename=shpfilename, driver = 'ESRI Shapefile', crs=ESRI_WKT)


# from keplergl import KeplerGl
# map_2 = KeplerGl()
# filename = "data"
# map_2.add_data(data=gpd.read_file("../SHPfiles/travelepisodew.shp"), name=filename)
# map_2

